In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
%matplotlib inline
from sklearn import preprocessing
import netron
from keras.models import load_model
import pickle

In [ ]:
#set the header list
header_list = ["Frame","Face","keypoint","Body","1_x","1_y","1_accuracy","2_x","2_y","2_accuracy","3_x","3_y","3_accuracy","4_x","4_y","4_accuracy","5_x","5_y","5_accuracy","6_x","6_y","6_accuracy","7_x","7_y","7_accuracy","8_x","8_y","8_accuracy","9_x","9_y","9_accuracy",
               "10_x","10_y","10_accuracy","11_x","11_y","11_accuracy","12_x","12_y","12_accuracy","13_x","13_y","13_accuracy","14_x","14_y","14_accuracy","15_x","15_y","15_accuracy","16_x","16_y","16_accuracy","17_x","17_y","17_accuracy","18_x","18_y","18_accuracy","19_x","19_y","19_accuracy",
               "20_x","20_y","20_accuracy","21_x","21_y","21_accuracy","22_x","22_y","22_accuracy","23_x","23_y","23_accuracy","24_x","24_y","24_accuracy","25_x","25_y","25_accuracy"]

df = pd.read_csv('C:/Users/CASALAB/Desktop/progetto_CASA/merged_json_csv.csv', names=header_list)
#set lists with frame data extracted with dlib and openpose
frame=df['Frame']
face=df['Face']
body=df['Body']

le = preprocessing.LabelEncoder()
# Converting string labels into numbers.
face_encoded=le.fit_transform(face)
body_encoded=le.fit_transform(body)

print ("Frame:", frame)
print ("Face:", face_encoded)
print("Body:", body_encoded)

#create the list with images
train_image = []
for i in tqdm(range(frame.shape[0])):
    img = image.load_img('C:/Users/CASALAB/Desktop/progetto_CASA/codice/prova3/'+frame[i][:6]+'/'+frame[i][-4:]+'.jpg',target_size=(150,150,3))
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)
X.shape



In [ ]:
#function to print the accuracy
def print_acc(hist):
    epochs = range(1, len(hist['accuracy'])+1)
    plt.plot(epochs, hist['accuracy'], 'b', label='Training acc')    
    plt.plot(epochs, hist['val_accuracy'], 'r', label='Validation acc')
    plt.title('Training and validation acc')
    plt.xlabel('Epochs')
    plt.ylabel('Acc')
    plt.legend()
    plt.show()

In [ ]:
#function to print the loss
def print_loss(hist):
    epochs = range(1, len(hist['loss'])+1)
    plt.plot(epochs, hist['loss'], 'b', label='Training loss')
    plt.plot(epochs, hist['val_loss'], 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

In [ ]:
#create the y variable containing features
features=list(zip(face_encoded,body_encoded))
y=np.array(features)
y.shape

In [ ]:
#split label and feature into two array containing training(80%) and validation(20%) data
train_pct_index = int(0.8 * len(X))
X_train, X_test = X[:train_pct_index], X[train_pct_index:]
y_train, y_test = y[:train_pct_index], y[train_pct_index:]

In [ ]:
from sklearn.utils import shuffle
X_train , y_train = shuffle ( X_train , y_train , random_state =0)
X_test , y_test = shuffle ( X_test , y_test , random_state =0)

In [ ]:
#create model layer
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5, 5), activation="relu", input_shape=(150,150,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=64, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=128, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='sigmoid'))
print('done')

In [ ]:
#save the model and print a resume
model.save('Modello.hdf5')
model.summary()

In [ ]:
#start netron software
netron.start('Modello.hdf5')

In [ ]:
#compile the model with optimizer and loss function
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
#training the model with training set and validate with validation set
dense_hist = model.fit(X_train, y_train, epochs=25, validation_data=(X_test, y_test), batch_size=64)

In [ ]:
#save the trained model
model.save('ModelloAllenato.hdf5')

In [ ]:
with open('trainHistoryDict', 'wb') as file_pi:
        pickle.dump(dense_hist, file_pi)

In [ ]:
#load the train story
history = pickle.load(open('trainHistoryDict', "rb"))

In [ ]:
dense_hist = dense_hist.history

In [ ]:
#print the accuracy
print_acc(dense_hist)

In [ ]:
#print the loss
print_loss(dense_hist)